In [149]:
import pandas as pd 
import dask.dataframe as dd
# import pyarrow as pa
import numpy as np
import tqdm
from datetime import datetime as dt
# from tsfresh.feature_extraction import extract_features
from pymongo import MongoClient
from sklearn.model_selection import train_test_split
from scipy import stats
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot
import scipy
import seaborn
import jieba
# jieba.enable_parallel(4)

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import svm
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import confusion_matrix,f1_score
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.preprocessing import StandardScaler

import lightgbm as lgbm

In [150]:
collection = MongoClient('52.68.95.18:27017')['BTCUSD_order_book']['binance_order_book']

In [151]:
order_book_df = pd.DataFrame(list(collection.find({})))

ServerSelectionTimeoutError: 52.68.95.18:27017: timed out

In [ ]:
order_book_df.head(3)

In [ ]:
order_book_df.drop(['market_pair', 'datetime', 'timestamp'], axis=1 , inplace=True)

In [ ]:
ticker_collection = MongoClient('52.68.95.18:27017')['BTCUSD_order_book']['binance_ticker']

In [ ]:
ticker_df = pd.DataFrame(list(ticker_collection.find({})))

In [ ]:
for column in ['askPrice', 'askQty', 'bidPrice', 'bidQty']:
    ticker_df[column] = ticker_df[column].astype(float)

In [ ]:
ticker_df.head(3)

In [ ]:
ticker_df.drop(['market_pair', 'symbol', 'timestamp', 'datetime','closeTime','count','firstId','lastId','openTime','weightedAvgPrice'], axis=1, inplace=True)

In [ ]:
ticker_df['askBidMean'] = (ticker_df['askPrice'] + ticker_df['bidPrice'])/2

In [ ]:
train_df = pd.merge(order_book_df, ticker_df, on='updatetime')

In [ ]:
train_df['datetime_hour'] = train_df['updatetime'].apply(lambda x: dt.strftime(dt.strptime(x,"%Y-%m-%d %H:%M:%S"),"%Y-%m-%d %H:%M"))

In [ ]:
for column in ['buy_ratio', 'buy_volume', 'last_price',
       'sell_ratio', 'sell_volume', 'total_buy_btc', 'total_sell_btc',
       'askPrice', 'askQty', 'bidPrice', 'bidQty',
       'highPrice', 'lastPrice', 'lastQty', 'lowPrice', 'openPrice',
       'prevClosePrice', 'priceChange', 'priceChangePercent', 'quoteVolume',
       'volume', 'askBidMean']:
    train_df[column] = train_df[column].astype(float)

In [ ]:
groupby = train_df.groupby("datetime_hour").mean()
groupby1 = train_df.groupby("datetime_hour").max().drop(['asks', 'bids', 'updatetime','_id_y', '_id_x'], axis=1)
groupby2 = train_df.groupby("datetime_hour").min().drop(['asks', 'bids', 'updatetime','_id_y', '_id_x'], axis=1)
groupby3 = train_df.groupby("datetime_hour").skew()
groupby4 = train_df.groupby("datetime_hour").median()
groupby5 = train_df.groupby("datetime_hour").std()
y = train_df.groupby("datetime_hour").last()['askBidMean']

In [ ]:
cleanData = pd.concat([groupby],axis=1)
# cleanData = groupby.drop(['asks', 'bids', '_id_y', 'updatetime','_id_x'], axis=1)
# cleanData = cleanData.drop(['asks', 'bids', 'updatetime','_id_y', '_id_x'], axis=1)

In [ ]:
x_window, y_window = 1, 1200

In [ ]:
x = np.zeros((len(cleanData)-x_window-y_window+1,x_window, len(cleanData.columns)))

In [ ]:
y_max = np.zeros(len(cleanData)-x_window-y_window+1)
y_min = np.zeros(len(cleanData)-x_window-y_window+1)
y_mean = np.zeros(len(cleanData)-x_window-y_window+1)
y_ma = np.zeros(len(cleanData)-x_window-y_window+1)

In [ ]:
cleanDataX = np.asarray(cleanData)
cleanDataY = np.asarray(y)

In [ ]:
# scaler = StandardScaler()
# scaler.fit(cleanDataX)
# cleanDataX = scaler.transform(cleanDataX)

In [ ]:
for index, i in enumerate(cleanDataX):
    x[index] = cleanDataX[index:index+x_window]
    y_mean[index] = cleanDataY[(index+x_window+y_window)-1]
    y_max[index] = cleanDataY[(index+x_window): (index+x_window+y_window)].max()
    y_min[index] = cleanDataY[(index+x_window): (index+x_window+y_window)].min()
    
    if index+x_window+y_window == len(cleanDataX):
        break

In [ ]:
x[500]

In [ ]:
clf.predict(x[500])

In [ ]:
y_mean[500]

In [ ]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Activation, Flatten, Dropout, GRU, Input
from keras.layers import Embedding
from keras.optimizers import Adam
from keras.models import Model

In [ ]:
stratified_folder = KFold(n_splits=4, random_state=42, shuffle=True) 
for train_index, test_index in stratified_folder.split(x,y_mean): 
    X_train, X_test = x[train_index].reshape(x[train_index].shape[0],-1), x[test_index].reshape(x[test_index].shape[0],-1)
    y_train, y_test = y_mean[train_index], y_mean[test_index] 
    lgb_params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'mse',
        'subsample': .7,
        'colsample_bytree': .8,
        'reg_alpha': .05,
        'reg_lambda': .05,
        'min_split_gain': 0.0001,
        'min_child_weight': 1,
        'silent':True,
        'verbosity':-1,
        'nthread':3
    }

    clf = lgbm.LGBMRegressor(**lgb_params,learning_rate=0.05,
                                    n_estimators=1000,max_depth=2)
    clf.fit(
        X_train, y_train,
        eval_set=[(X_train, y_train),(X_test, y_test)],
        verbose=10,
        eval_metric='rmse',
        early_stopping_rounds=100
    )
    pred = clf.predict(X_test, num_iteration=clf.best_iteration_)
    print("predict ")
    print(pred)
    print("true ")
    print(y_test)
#     print(f1_score(pred, y_test, average='macro'))
#     print(confusion_matrix(pred, y_test))

In [108]:
stratified_folder = KFold(n_splits=4, random_state=0, shuffle=True) 
for train_index, test_index in stratified_folder.split(x,y_mean): 
    X_train, X_test = x[train_index], x[test_index]
    y_train_mean, y_test_mean = y_mean[train_index], y_mean[test_index]
    y_train_max, y_test_max = y_max[train_index], y_max[test_index] 
    y_train_min, y_test_min = y_min[train_index], y_min[test_index] 
    
    input_layer = Input(shape=(X_train.shape[1],X_train.shape[2]))
    hidden_layer = GRU(32, return_sequences=True, activation='tanh')(input_layer)
    hidden_layer = Flatten()(hidden_layer)
    
    mean_hidden_layer = Dense(2048, activation='relu')(hidden_layer)
    mean_hidden_layer = Dropout(0.1)(mean_hidden_layer)
    
    mean_hidden_layer = Dense(2048, activation='relu')(mean_hidden_layer)
    mean_hidden_layer = Dropout(0.1)(mean_hidden_layer)
#     max_hidden_layer = Dense(256, activation='relu')(hidden_layer)
#     max_hidden_layer = Dropout(0.1)(max_hidden_layer)
    
#     min_hidden_layer = Dense(256, activation='relu')(hidden_layer)
#     min_hidden_layer = Dropout(0.1)(min_hidden_layer)
    
    output_layer_1 = Dense(1,activation='linear')(mean_hidden_layer)
#     output_layer_2 = Dense(1,activation='linear')(max_hidden_layer)
#     output_layer_3 = Dense(1,activation='linear')(min_hidden_layer)
    
    opt = Adam(lr=0.01)
    
    ml_model = Model(inputs=input_layer, outputs=[output_layer_1])
    ml_model.compile(optimizer=opt, loss='mse')
    ml_model.fit(X_train, [y_train_mean], epochs=1000, verbose=1, batch_size=8192)
    
    pred = ml_model.predict(X_test, batch_size=8192)
    break
#     print(f1_score(pred, y_test, average='macro'))
#     print(confusion_matrix(pred, y_test))

Epoch 1/1000
6108/6108 [==============================] - 4s 588us/step - loss: 124979640.0000
Epoch 2/1000


KeyboardInterrupt: 